In [1]:
# Pathways
import os
import sys
from pathlib import Path

# Automatically set the project path to the current working directory
project_path = Path.cwd()
os.chdir(project_path)
sys.path.insert(0, str(project_path))

print(f"Current directory: {os.getcwd()}")

Current directory: c:\Users\joshu\OneDrive\Documents\1 Work\Bank of England NLP\Bank-of-England-NLP-on-Earnings-Calls


In [2]:
# -- Define prompt text for extraction --
prompt_text = {
    "role": "system",
    "content": f"""
        Extract structured JSON objects from a transcript while maintaining speech order. 
        Each question and answer should be a distinct JSON object. Exclude Operator responses, greetings, and farewells. 
        If multiple people respond, capture each as a separate answer object.

        Each JSON object should include:
        - Type (Question/Answer)
        - Text (Spoken content)
        - Page (Transcript page number)
        - Person (Speaker’s name)
        - Role (Speaker’s role, e.g., Analyst, CFO)
        - Affiliation (Organization the speaker represents)
        - Topics (Up to 5, each with a brief justification)
        - Sentiment Analysis (Score: -1 to +1, Category, Justification)

        Ensure strict JSON compliance without extra punctuation or spacing errors.

    **Example JSON Structure:**

    {{
        "Type": "Question",
        "Text": "Can you elaborate on how JPMorgan Chase plans to address potential capital deficits while maintaining growth?",
        "Page": 3,
        "Person": "John McDonald",
        "Role": "Analyst",
        "Affiliation": "Truist Securities, Inc.",
        "Topics": [
            {{"Name": "Capital Management", "Justification": "Addresses capital reserves and growth balance."}},
            {{"Name": "Risk Management", "Justification": "Focuses on mitigating financial risk."}}
        ],
        "Sentiment": {{
            "Score": 0.1,
            "Category": "Neutral",
            "Justification": "Neutral inquiry seeking information."
        }}
    }}

    {{
        "Type": "Answer",
        "Text": "We have a comprehensive capital strategy that ensures we meet regulatory requirements while supporting business growth...",
        "Page": 3,
        "Person": "Jeremy Barnum",
        "Role": "Chief Financial Officer",
        "Affiliation": "JPMorgan Chase",
        "Topics": [
            {{"Name": "Capital Management", "Justification": "Balances compliance with growth."}},
            {{"Name": "Risk Management", "Justification": "Acknowledges market volatility and adaptability."}},
            {{"Name": "Regulatory Compliance", "Justification": "Ensures strategies meet regulatory needs."}}
        ],
        "Sentiment": {{
            "Score": -0.2,
            "Category": "Slightly Negative",
            "Justification": "Acknowledgment of challenges introduces slight negativity."
        }}
    }}

    {{
        "Type": "Answer",
        "Text": "To add, our strong liquidity position and diversified revenue streams provide a buffer against potential deficits...",
        "Page": 4,
        "Person": "Jamie Dimon",
        "Role": "Chairman & CEO",
        "Affiliation": "JPMorgan Chase",
        "Topics": [
            {{"Name": "Capital Management", "Justification": "Emphasizes liquidity and revenue diversification."}},
            {{"Name": "Resilience", "Justification": "Highlights strength against market fluctuations."}}
        ],
        "Sentiment": {{
            "Score": 0.3,
            "Category": "Slightly Positive",
            "Justification": "Focus on strengths conveys a slightly positive outlook."
        }}
    }}

    {{
        "Type": "Question",
        "Text": "How do you see current interest rate trends impacting your capital strategy?",
        "Page": 4,
        "Person": "Jane Doe",
        "Role": "Analyst",
        "Affiliation": "Goldman Sachs",
        "Topics": [
            {{"Name": "Interest Rates", "Justification": "Examines interest rate impact on capital management."}}
        ],
        "Sentiment": {{
            "Score": 0.0,
            "Category": "Neutral",
            "Justification": "Factual, unbiased inquiry."
        }}
    }}

    {{
        "Type": "Answer",
        "Text": "Rising interest rates can increase funding costs, but they also create opportunities to enhance net interest income...",
        "Page": 5,
        "Person": "Jeremy Barnum",
        "Role": "Chief Financial Officer",
        "Affiliation": "JPMorgan Chase",
        "Topics": [
            {{"Name": "Interest Rates", "Justification": "Acknowledges both challenges and opportunities."}}
        ],
        "Sentiment": {{
            "Score": 0.2,
            "Category": "Slightly Positive",
            "Justification": "Recognizes cost increases but emphasizes opportunities."
        }}
    }}
    """
}

In [3]:
import os
import openai
import tiktoken
import pandas as pd
import json
import time
from dotenv import load_dotenv
import PyPDF2
import mlflow
import mlflow.pyfunc
from datetime import datetime
from pathlib import Path

# -- Environment and API setup --
load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Use tokenizer for token counting (ensure this matches your model)
tokenizer = tiktoken.get_encoding("o200k_base")

# Set up MLflow experiment
mlflow.set_experiment("Topic_Modeling_Extraction")

def count_tokens(text):
    """Returns the number of tokens in a given text."""
    return len(tokenizer.encode(text))

def extract_roles_and_affiliations(transcript_data):
    """Calls OpenAI API to extract job roles and company names from structured PDF transcript data."""
    # Convert DataFrame to JSON format
    json_text = transcript_data.to_json(orient='records')
    start_time = time.time()
    token_count = count_tokens(json_text)

    prompt = {
        "model": "gpt-4o",
        "messages": [
            {"role": "system", "content": f"**Instruction**: {prompt_text}"},
            {"role": "user", "content": f"**Input Data**: {json_text}"}
        ],
        "temperature": 0.2,
    }

    with mlflow.start_run():
        try:
            response = client.chat.completions.create(**prompt)
            latency = time.time() - start_time

            # Log parameters and metrics
            mlflow.log_param("model", "gpt-4o")
            mlflow.log_param("temperature", 0.2)
            mlflow.log_metric("latency_seconds", latency)
            mlflow.log_metric("token_count", token_count)
            mlflow.log_metric("response_length", count_tokens(response.choices[0].message.content))

            # Log artifacts (input and output)
            mlflow.log_text(json_text, "input_transcript.json")
            mlflow.log_text(response.choices[0].message.content, "output_extracted.json")

            return response.choices[0].message.content.strip()
        except Exception as e:
            mlflow.log_param("error", str(e))
            print(f"API Error: {e}")
            return None

# -- Process each PDF in the input folder --
input_folder = Path("1_data")
output_folder = Path("3_processed_json")
output_folder.mkdir(parents=True, exist_ok=True)

for pdf_file_path in input_folder.glob("*.pdf"):
    print(f"\nProcessing PDF: {pdf_file_path.name}")
    extracted_data = []
    with open(pdf_file_path, "rb") as pdf_file:
        reader = PyPDF2.PdfReader(pdf_file)
        for page_number, page in enumerate(reader.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                extracted_data.append({"Page Number": page_number, "Text": page_text})
    
    # Convert the list of page data into a DataFrame
    transcript_data = pd.DataFrame(extracted_data)
    extracted_json = extract_roles_and_affiliations(transcript_data)
    
    if extracted_json:
        output_file_name = f"processed_{pdf_file_path.stem}.json"
        output_file = output_folder / output_file_name
        with open(output_file, "w", encoding="utf-8") as f:
            f.write(extracted_json)
        print(f"Processing completed successfully. Output saved to: {output_file}")
    else:
        print(f"Processing failed for {pdf_file_path.name}")



Processing PDF: jpm-1q24-earnings-call-transcript.pdf
Processing completed successfully. Output saved to: 3_processed_json\processed_jpm-1q24-earnings-call-transcript.json

Processing PDF: jpm-2q24-earnings-call-transcript.pdf
Processing completed successfully. Output saved to: 3_processed_json\processed_jpm-2q24-earnings-call-transcript.json

Processing PDF: jpm-3q24-earnings-call-transcript.pdf
Processing completed successfully. Output saved to: 3_processed_json\processed_jpm-3q24-earnings-call-transcript.json

Processing PDF: jpm-4q24-earnings-call-transcript.pdf
Processing completed successfully. Output saved to: 3_processed_json\processed_jpm-4q24-earnings-call-transcript.json

Processing PDF: Q1_Transcript-Analyst-Call-25-April-2024.pdf
Processing completed successfully. Output saved to: 3_processed_json\processed_Q1_Transcript-Analyst-Call-25-April-2024.json

Processing PDF: Q2_Transcript-Analyst-Call-24-July-2024-For-Upload.pdf
Processing completed successfully. Output saved t

In [4]:
import json
from IPython.display import display

# Define the folder containing the processed JSON files
output_folder = Path("3_processed_json")
json_files = list(output_folder.glob("*.json"))

# Dictionary to store each transcript DataFrame separately
transcript_dfs = {}

for json_file in json_files:
    with open(json_file, "r", encoding="utf-8") as file:
        file_content = file.read().strip()

    # Remove Markdown-style JSON code block markers if present
    if file_content.startswith("```json") and file_content.endswith("```"):
        file_content = file_content[7:-3].strip()

    # Ensure the JSON objects are wrapped in a proper list format
    file_content_cleaned = "[" + file_content.replace("}\n{", "},\n{") + "]"
    
    try:
        json_data = json.loads(file_content_cleaned)
        df = pd.DataFrame(json_data)
        transcript_dfs[json_file.name] = df  # store the DataFrame
        
        # Display the first few rows of each transcript's DataFrame
        print(f"DataFrame for {json_file.name}:")
        display(df.head())
    except Exception as e:
        print(f"Error processing {json_file.name}: {e}")


DataFrame for processed_jpm-1q24-earnings-call-transcript.json:


,Type,Text,Page,Person,Role,Affiliation,Topics,Sentiment
0,Question,"Jamie, could you talk through the decision to ...",4,Betsy L. Graseck,Analyst,Morgan Stanley & Co. LLC,"[{'Name': 'Dividend Policy', 'Justification': ...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
1,Answer,"So, Betsy, before I answer the question, I wan...",4,Jamie Dimon,Chairman & CEO,JPMorgan Chase & Co.,"[{'Name': 'Dividend Policy', 'Justification': ...","{'Score': 0.2, 'Category': 'Slightly Positive'..."
2,Question,"Jeremy, can you speak to the trends you're see...",6,Jim Mitchell,Analyst,Seaport Global Securities LLC,"[{'Name': 'Deposit Trends', 'Justification': '...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
3,Answer,"Yeah. Good question, Jim. I think the simplest...",6,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase & Co.,"[{'Name': 'Deposit Trends', 'Justification': '...","{'Score': 0.1, 'Category': 'Slightly Positive'..."
4,Question,"Jeremy, you had mentioned at a conference earl...",7,John McDonald,Analyst,Autonomous Research,"[{'Name': 'Credit Reserves', 'Justification': ...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."


DataFrame for processed_jpm-2q24-earnings-call-transcript.json:


,Type,Text,Page,Person,Role,Affiliation,Topics,Sentiment
0,Question,"So, wanted to start off with a question on cap...",4,Steven Chubak,Analyst,Wolfe Research LLC,"[{'Name': 'Capital Management', 'Justification...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
1,Answer,"Right. Okay. Thanks, Steve. And actually, befo...",4,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase,"[{'Name': 'Capital Management', 'Justification...","{'Score': -0.1, 'Category': 'Slightly Negative..."
2,Question,"Maybe just for my follow-up on NII, you've bee...",5,Steven Chubak,Analyst,Wolfe Research LLC,"[{'Name': 'Net Interest Income', 'Justificatio...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
3,Answer,"Yeah. Interesting question, Steve. So, let's t...",5,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase,"[{'Name': 'Deposit Management', 'Justification...","{'Score': -0.1, 'Category': 'Slightly Negative..."
4,Question,Can you give an update on the stress capital b...,5,Saul Martinez,Analyst,"HSBC Securities (USA), Inc","[{'Name': 'Regulatory Compliance', 'Justificat...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."


DataFrame for processed_jpm-3q24-earnings-call-transcript.json:


...

DataFrame for processed_Q2_Transcript-Analyst-Call-24-July-2024-For-Upload.json:


,Type,Text,Page,Person,Role,Affiliation,Topics,Sentiment
0,Question,"So, the first question is on revenues. I guess...",15,Chris Hallam,Analyst,Goldman Sachs,"[{'Name': 'Revenue Forecast', 'Justification':...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
1,Question,"And then second, could you give us an update o...",15,Chris Hallam,Analyst,Goldman Sachs,"[{'Name': 'Regulatory Compliance', 'Justificat...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
2,Answer,"Thank you, Chris. It’s Christian. Thank you ve...",16,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,"[{'Name': 'Revenue Forecast', 'Justification':...","{'Score': 0.3, 'Category': 'Slightly Positive'..."
3,Answer,"And the nice thing is, if I now go over the ne...",16,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,"[{'Name': 'Revenue Forecast', 'Justification':...","{'Score': 0.2, 'Category': 'Slightly Positive'..."
4,Answer,"On the Corporate Bank, I think a number, which...",16,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,"[{'Name': 'Corporate Banking', 'Justification'...","{'Score': 0.2, 'Category': 'Slightly Positive'..."


DataFrame for processed_Q3-2024-Analyst-Call-Transcript-For-Upload.json:


,Type,Text,Page,Person,Role,Affiliation,Topics,Sentiment
0,Question,The first is on loan losses. When you talk abo...,17,Anke Reingen,Analyst,RBC,"[{'Name': 'Loan Loss Provisions', 'Justificati...","{'Score': -0.1, 'Category': 'Slightly Negative..."
1,Question,"And then, second, on the buybacks. Yes, great ...",17,Anke Reingen,Analyst,RBC,"[{'Name': 'Share Buybacks', 'Justification': '...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
2,Question,"So on the NII, could we come back, please, to ...",21,Flora Bocahut,Analyst,Barclays,"[{'Name': 'Net Interest Income', 'Justificatio...","{'Score': 0.1, 'Category': 'Neutral', 'Justifi..."
3,Question,And then the second question is on cost. I thi...,21,Flora Bocahut,Analyst,Barclays,"[{'Name': 'Cost Management', 'Justification': ...","{'Score': 0.0, 'Category': 'Neutral', 'Justifi..."
4,Question,"Firstly, on the US CRE, I just wanted to check...",32,Jeremy Sigee,Analyst,BNP Paribas Exane,"[{'Name': 'Commercial Real Estate', 'Justifica...","{'Score': 0.2, 'Category': 'Slightly Positive'..."


Error processing processed_Q4-2024-Analyst-Call-Transcript-For-Upload.json: Expecting value: line 1 column 2 (char 1)


In [ ]:
processed_dfs = {}

# Define column mapping for readability
column_mapping = {
    "Page": "Page Number",
    "Text": "Extracted Text",
    "Type": "Question/Answer",
    "Person": "Name",
    "Role": "Role",
    "Affiliation": "Affiliation",
    "Sentiment": "Sentiment",
    "Topics": "Topics"
}

for file_name, df in transcript_dfs.items():
    # If a "Page" column exists, convert it to integer and rename
    if "Page" in df.columns:
        df["Page"] = df["Page"].astype(int)
    
    available_columns = [col for col in column_mapping.keys() if col in df.columns]
    df_filtered = df[available_columns].rename(columns=column_mapping)
    df_filtered = df_filtered.sort_values(by="Page Number")
    
    # Expand sentiment details if available
    if "Sentiment" in df_filtered.columns:
        df_filtered["Sentiment Score"] = df_filtered["Sentiment"].apply(
            lambda x: x.get("Score") if isinstance(x, dict) else None
        )
        df_filtered["Sentiment Category"] = df_filtered["Sentiment"].apply(
            lambda x: x.get("Category") if isinstance(x, dict) else None
        )
        df_filtered["Sentiment Justification"] = df_filtered["Sentiment"].apply(
            lambda x: x.get("Justification") if isinstance(x, dict) else None
        )
    
    # Expand topics into separate columns (up to 5 topics)
    if "Topics" in df_filtered.columns:
        max_topics = 5
        for i in range(max_topics):
            df_filtered[f"Topic {i+1}"] = df_filtered["Topics"].apply(
                lambda x: x[i]["Name"] if isinstance(x, list) and len(x) > i else None
            )
            df_filtered[f"Topic {i+1} Justification"] = df_filtered["Topics"].apply(
                lambda x: x[i]["Justification"] if isinstance(x, list) and len(x) > i else None
            )
    
    # Optionally drop the original nested columns
    df_filtered = df_filtered.drop(columns=["Sentiment", "Topics"], errors="ignore")
    
    processed_dfs[file_name] = df_filtered
    
    # Display the processed DataFrame for each transcript
    print(f"Processed DataFrame for {file_name}:")
    display(df_filtered.head())


Processed DataFrame for processed_jpm-1q24-earnings-call-transcript.json:


,Page Number,Extracted Text,Question/Answer,Name,Role,Affiliation,Sentiment Score,Sentiment Category,Sentiment Justification,Topic 1,Topic 1 Justification,Topic 2,Topic 2 Justification,Topic 3,Topic 3 Justification,Topic 4,Topic 4 Justification,Topic 5,Topic 5 Justification
0,4,"Jamie, could you talk through the decision to ...",Question,Betsy L. Graseck,Analyst,Morgan Stanley & Co. LLC,0.1,Neutral,Neutral inquiry seeking clarification on finan...,Dividend Policy,Inquires about the rationale behind dividend a...,Capital Strategy,Seeks understanding of capital allocation deci...,None,None,None,None,None,None
1,5,We're earning a lot of money. Our capital cup ...,Answer,Jamie Dimon,Chairman & Chief Executive Officer,JPMorgan Chase & Co.,0.3,Slightly Positive,Positive outlook on earnings and capital posit...,Dividend Policy,Explains the reasoning behind dividend increase.,Capital Management,Discusses capital allocation and earnings dist...,None,None,None,None,None,None
2,6,"Jeremy, can you speak to the trends you're see...",Question,Jim Mitchell,Analyst,Seaport Global Securities LLC,0.0,Neutral,Seeks factual information on deposit trends.,Deposit Trends,Focuses on changes in deposit behavior.,Interest Rates,Implied impact of interest rates on deposit mi...,None,None,None,None,None,None
3,6,The simplest and best answer to that is: not r...,Answer,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase & Co.,0.1,Neutral,Provides a straightforward update on deposit t...,Deposit Trends,Confirms ongoing migration trends.,Interest Rates,Acknowledges rate-driven behavior in deposits.,None,None,None,None,None,None
4,8,"Jeremy, when you think about the outlook for t...",Question,Ebrahim H. Poonawala,Analyst,Bank of America Merrill Lynch,0.0,Neutral,Requests an assessment of economic and custome...,Economic Outlook,Seeks perspective on economic conditions.,Customer Health,Focuses on the financial health of customers.,None,None,None,None,None,None


Processed DataFrame for processed_jpm-2q24-earnings-call-transcript.json:


,Page Number,Extracted Text,Question/Answer,Name,Role,Affiliation,Sentiment Score,Sentiment Category,Sentiment Justification,Topic 1,Topic 1 Justification,Topic 2,Topic 2 Justification,Topic 3,Topic 3 Justification,Topic 4,Topic 4 Justification,Topic 5,Topic 5 Justification
0,4,"So, wanted to start off with a question on cap...",Question,Steven Chubak,Analyst,Wolfe Research LLC,0.0,Neutral,Seeks factual information about regulatory imp...,Capital Management,Discusses potential changes in capital require...,Regulatory Changes,Focuses on Basel III and GSIB surcharge revisi...,Financial Strategy,Inquires about the firm's strategy in response...,None,None,None,None
1,4,"Right. Okay. Thanks, Steve. And actually, befo...",Answer,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase,-0.1,Slightly Negative,Expresses skepticism about potential positive ...,Capital Management,Discusses the firm's capital strategy and ROTC...,Regulatory Changes,Mentions Basel III and GSIB surcharge consider...,Financial Strategy,Explains the firm's approach to capital return...,None,None,None,None
2,5,"Maybe just for my follow-up on NII, you've bee...",Question,Steven Chubak,Analyst,Wolfe Research LLC,0.0,Neutral,Requests information on NII trends and future ...,Net Interest Income,Focuses on NII trends and potential normalizat...,Deposit Management,Inquires about deposit pressures and their imp...,Financial Strategy,Seeks insight into future NII stabilization st...,None,None,None,None
3,5,"Yeah. Interesting question, Steve. So, let's t...",Answer,Jeremy Barnum,Chief Financial Officer,JPMorgan Chase,-0.1,Slightly Negative,Acknowledges ongoing challenges and uncertaint...,Net Interest Income,Discusses current NII trends and future outlook.,Deposit Management,Explains deposit balance pressures and their i...,Monetary Policy,Mentions Fed rate cuts and their influence on ...,None,None,None,None
4,5,"Hi, good morning. Thanks for taking my questio...",Question,Saul Martinez,Analyst,"HSBC Securities (USA), Inc",0.0,Neutral,Requests factual information about regulatory ...,Regulatory Compliance,Inquires about the stress capital buffer and F...,Capital Management,Focuses on potential adjustments to capital re...,Financial Strategy,Seeks insight into how regulatory changes migh...,None,None,None,None


Processed DataFrame for processed_jpm-3q24-earnings-call-transcript.json:


...

,Page Number,Extracted Text,Question/Answer,Name,Role,Affiliation,Sentiment Score,Sentiment Category,Sentiment Justification,Topic 1,Topic 1 Justification,Topic 2,Topic 2 Justification,Topic 3,Topic 3 Justification,Topic 4,Topic 4 Justification,Topic 5,Topic 5 Justification
0,15,"So, the first question is on revenues. I guess...",Question,Chris Hallam,Analyst,Goldman Sachs,0.1,Neutral,Neutral inquiry about revenue and fee projecti...,Revenue Projections,Inquires about confidence in revenue targets.,Fee Structure,Focuses on fee progression in different bank s...,None,None,None,None,None,None
1,15,"And then second, could you give us an update o...",Question,Chris Hallam,Analyst,Goldman Sachs,0.0,Neutral,Seeks factual update on regulatory discussions.,Regulatory Compliance,Concerns about ECB discussions on capital requ...,Leveraged Finance,Focuses on the outlook for the Leveraged Finan...,None,None,None,None,None,None
2,16,"Thank you, Chris. It’s Christian. Thank you ve...",Answer,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,0.3,Slightly Positive,Expresses satisfaction with current performance.,Revenue Projections,Confirms confidence in achieving revenue targets.,Business Performance,Highlights successful business delivery.,None,None,None,None,None,None
3,16,"On the Corporate Bank, I think a number, which...",Answer,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,0.2,Slightly Positive,Optimistic about Corporate Bank's performance.,Corporate Banking,Discusses revenue expectations for Corporate B...,Net Interest Income,Mentions better-than-expected NII performance.,None,None,None,None,None,None
4,17,"On the Investment Bank side, I absolutely beli...",Answer,Christian Sewing,Chief Executive Officer,Deutsche Bank AG,0.3,Slightly Positive,Confident in the Investment Bank's future perf...,Investment Banking,Focuses on the positive trajectory of the Inve...,Market Share,Highlights gains in market share.,None,None,None,None,None,None


Processed DataFrame for processed_Q3-2024-Analyst-Call-Transcript-For-Upload.json:


,Page Number,Extracted Text,Question/Answer,Name,Role,Affiliation,Sentiment Score,Sentiment Category,Sentiment Justification,Topic 1,Topic 1 Justification,Topic 2,Topic 2 Justification,Topic 3,Topic 3 Justification,Topic 4,Topic 4 Justification,Topic 5,Topic 5 Justification
0,17,The first is on loan losses. When you talk abo...,Question,Anke Reingen,Analyst,RBC,-0.1,Slightly Negative,Expresses concern over upward revisions and se...,Loan Loss Provisions,Focuses on expected changes in loan loss provi...,Financial Guidance,Questions accuracy of previous financial guida...,Postbank Integration,Inquires about the impact of Postbank integrat...,None,None,None,None
1,17,"And then, second, on the buybacks. Yes, great ...",Question,Anke Reingen,Analyst,RBC,0.0,Neutral,Seeks factual information without expressing o...,Share Buybacks,Seeks details on share buyback plans.,Financial Strategy,Inquires about strategic financial decisions.,Future Projections,Asks about future financial decisions and thei...,None,None,None,None
2,21,"So on the NII, could we come back, please, to ...",Question,Flora Bocahut,Analyst,Barclays,0.1,Slightly Positive,Acknowledges reassurance from hedging strategy...,Net Interest Income,Focuses on changes in NII and its implications.,Hedging Strategy,Inquires about the impact of hedging strategie...,Interest Rates,Considers the impact of interest rate changes ...,None,None,None,None
3,21,And then the second question is on cost. I thi...,Question,Flora Bocahut,Analyst,Barclays,0.0,Neutral,Seeks factual information on cost management s...,Cost Management,Focuses on achieving cost reduction targets.,Financial Guidance,Seeks clarification on cost guidance and confi...,None,None,None,None,None,None
4,23,And then second question is related to your 20...,Question,Kian Abouhossein,Analyst,JPMorgan,-0.1,Slightly Negative,Expresses concern over ambitious revenue growt...,Revenue Growth,Focuses on revenue growth targets and assumpti...,Economic Environment,Considers the impact of the German economic en...,Financial Projections,Inquires about assumptions underlying future r...,None,None,None,None


Processed DataFrame for processed_transcript_test.json:


,Page Number,Extracted Text,Question/Answer,Name,Role,Affiliation,Sentiment Score,Sentiment Category,Sentiment Justification,Topic 1,Topic 1 Justification,Topic 2,Topic 2 Justification,Topic 3,Topic 3 Justification,Topic 4,Topic 4 Justification,Topic 5,Topic 5 Justification
0,15,"Two questions, first of all, on the revenues. ...",Question,Kian Abouhossein,Analyst,JP Morgan,0.0,Neutral,The question is factual and seeks detailed inf...,Revenue Targets,Inquires about the breakdown of revenue target...,Cost Management,Seeks clarification on cost assumptions and ta...,Bank Levies,Asks about assumptions related to bank levies.,None,None,None,None
1,15,"Look, to your first question, let me tackle th...",Answer,Christian Sewing,Chief Executive Officer,Deutsche Bank,0.3,Slightly Positive,Optimistic about revenue growth and investment...,Revenue Growth,Discusses progress towards revenue targets.,Net Interest Income,Mentions positive performance of NII.,Investment Returns,Highlights success in fee-generating investments.,None,None,None,None
2,19,"Yes, thank you very much for taking my questio...",Question,Anke Reingen,Analyst,RBC,0.0,Neutral,The question is factual and seeks detailed inf...,Capital Management,Inquires about CET1 ratio and buyback potential.,Regulatory Impact,Asks about regulatory headwinds and impacts.,Loan Loss Provisions,Seeks clarification on loan loss expectations ...,None,None,None,None
3,19,"Thanks, Anke. So, look, the target that we’ve ...",Answer,James von Moltke,Chief Financial Officer,Deutsche Bank,0.1,Neutral,Provides a factual response with a slight posi...,Capital Management,Discusses CET1 ratio targets and capital optim...,Regulatory Compliance,Mentions Basel III impacts and regulatory targ...,None,None,None,None,None,None
4,21,"Yes, good morning. Thanks for taking my questi...",Question,Nicolas Payen,Analyst,Kepler Cheuvreux,0.0,Neutral,The question is factual and seeks detailed inf...,Net Interest Income,Seeks clarification on conditions affecting NI...,Share Buybacks,Inquires about plans and regulatory discussion...,None,None,None,None,None,None


In [ ]:
# To view a specific transcript's DataFrame:
if "processed_dfs" not in globals():
    print("⚠️ processed_dfs not defined. Please run the cell that creates it first.")
else:
    file_to_view = "processed_jpm-4q24-earnings-call-transcript.json"  # change this as needed
    if file_to_view in processed_dfs:
        display(processed_dfs[file_to_view])
    else:
        print(f"No DataFrame found for {file_to_view}. Available files: {list(processed_dfs.keys())}")


⚠️ processed_dfs not defined. Please run the cell that creates it first.
